# Homework 7 - Adrian Mead
## Data Loading and Integration

### Assignment:
1. Submit a jupyter notebook with functions that load each year of the train accident data into a pandas dataframe.
2. This dataframe should also include a variable, Narrative, that integrates the separate narrative fields into one column.

In [32]:
# Packages for loading, cleaning, visualization, and analysis

# Data
import pandas as pd
import numpy as np
import scipy as sp
import os
import string as st

1. Functions that load each year of the train accident data into a pandas dataframe.

In [18]:
# I have my own function that will pull each year of the train data in a panda df
# You pass a directory and it returns a list of pandas dataframes
def getListDataFrames(path):
    # initialize an empty list
    csvs = list()
    # Get all the files in the directory
    all_files = os.listdir(path)
    # Iterate across each of the files in the Data folder
    for fileName in all_files:
        # Get the full path name
        fileToPull = path + fileName
        # Read in the data at that location
        filesOfInterest = pd.read_csv(fileToPull, index_col = False, low_memory = False)
        csvs.append(filesOfInterest)
    return csvs

# Here is the list of 16 pandas dataframes corresponding to 2001 thru 2016
listOfAccidents = getListDataFrames('/Users/mead/Fall2017/DonBrown-DS6001/InClass1/Data/')

In [20]:
# How many elements are in the list
print("Length of the list: " + str(len(listOfAccidents)))

# Now to append all the list elements together into one pandas df
acc_df = pd.concat(listOfAccidents, ignore_index = True)
print("Shape of the 16 years of accidents data: " + str(acc_df.shape))

Length of the list: 16
Shape of the 16 years of accidents data: (51623, 153)


#2. This dataframe should also include a variable, Narrative, that integrates the separate narrative fields into one column.

In [23]:
# Produce a function to add all of the Narratives together
# Takes the index of the NARR1 column as input alongside the dataframe of interest. 
# It outputs the full Narrative column
def combineNARR(NARR1index, dataFrme):
# Initialize an empty list of all narratives
    allAccs = list()
    # Iterate across every row
    for acc in range(len(dataFrme.index)):
        # Initialize an empty list of each row's narrative
        fullNarr = list()
        # Iterate across every NARR column
        for narr in range(15):
            index = narr + NARR1index[0]
            # Keep track of the value in that NARR column for that row
            narrElement = str(dataFrme.iloc[acc, index])
            fullNarr.append(narrElement)
        allAccs.append(fullNarr)
    return allAccs

# This is the which function which keeps track of where NARR1 is
which = lambda lst:list(np.where(lst)[0])
NARR1index = which([name == 'NARR1' for name in acc_df.columns])

# Run the function -- output is a list of lists
newColumn = combineNARR(NARR1index, acc_df)

# Combine all the NARR together for each row. Make sure to remove the NAs
Narratives = [''.join([sentence for sentence in NARR if sentence != 'nan']) for NARR in newColumn]
# Add this as a new column for the accidents df
acc_df['Narrative'] = pd.Series(Narratives, index=acc_df.index)

In [29]:
# Output for the first 5 narratives
print("The first 5 narratives: " + str(Narratives[0:4]))

The first 5 narratives: ['UNITS 231-281(BACK TO BACK)  WERE COMING INTO UP DEISEL SHOP  WHEN THE LEFT WHEEL OF 281 RODE OVER RECENTLY REPAIRED SWITCH PLATE AND DERAILED. THE CAUSE WAS DETERMINED TO BE THE TRACK TELEMETRY IN THAT IT WAS TOO SHARP OF A CURVE.', 'ENGINE 286 CAUGHT FIRE AT THE SPRINGFIELD, MA STATION DUE TO BEARINGS IN MAIN GENERATOR LET GO.', 'TRAIN NO.#4 WITH ENGS 83/11/90/44 AND 11 CARS DERAILED 2 DEADHEAD CARS, C/44834 AND C/9639, WHILE MAKING A SHOVING MOVE ONTO TRACK 28.  THE DERAILMENT WAS DUE TO HIGH BUFF FORCES CAUSED JACKKNIFING OFDEADHEADING AMFLEET CAR 44834 LOCATED DIRECTLY BEHIND ENGINES DUE TO EXCESSIVE AMPERAGE GENERATED BY FOUR P42 LOCOMOTIVES SHOVING TRAIN AGAINST AN APPROXIMATELY 15-POUND BRAKE REDUCTION.', 'WHILE SHOVING TRAIN 624 SOUTH ON #30 TRACK AT PENN COACH YARD, CONDUCTOR FAILED TO STOP FOR A DERAILAND 2 CARS DERAILED .']


In [30]:
# Also going to remove the old NARR columns
# Drop old narrative columns
narrList = []
for i in range(0,15):
    a = str(i + 1)
    narrList.append('NARR' + a)
acc_df.drop(narrList, axis = 1, inplace = True)

In [31]:
# The output of the new Narrative column
print(acc_df[['Narrative']].head())

                                           Narrative
0  UNITS 231-281(BACK TO BACK)  WERE COMING INTO ...
1  ENGINE 286 CAUGHT FIRE AT THE SPRINGFIELD, MA ...
2  TRAIN NO.#4 WITH ENGS 83/11/90/44 AND 11 CARS ...
3  WHILE SHOVING TRAIN 624 SOUTH ON #30 TRACK AT ...
4  TRAIN 786 WAS STRUCK BY A FALLING TREE SOUTH O...
